In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)

from doe_gensplit.doe import *
from doe_gensplit.init import *
from doe_gensplit.utils import *
from doe_gensplit.validation import *

## Design setup

In [2]:
# The model
model = pd.read_excel(f'..\\assets\\Grass2022_max_model.xlsx', index_col=0).to_numpy()
columns = ['D', 'C', 'B', 'A', 'LOC', 'Cap', 'RPM']

# The factor columns
factors = np.array([
    [3, 1],         # D
    [2, 1],         # C
    [1, 1],         # B
    [1, 1],         # A
    [0, 1],         # LOC
    [0, 1],         # Capacity
    [0, 1],         # RPM
])

# Plot sizes
plot_sizes = np.array([3, 3, 3, 2], dtype=np.int64)

# Compute V-matrix (observation correlation matrix)
V = obs_var(plot_sizes)

# Encode the model
model_enc = encode_model(model, factors)

# Validate the model
validate_model(model, factors)

terms = ['icept', 'D', 'C', 'C^2', 'D \\times C', 'B', 'A', 'D \\times B', 'D \\times A',
         'C \\times B', 'C^2 \\times B', 'C \\times A', 'C^2 \\times A', 'B \\times A',
         'LOC', 'LOC^2', 'Cap', 'Cap^2', 'RPM', 'RPM^2', 'D \\times LOC', 'D \\times Cap',
         'D \\times RPM', 'C \\times LOC', 'C^2 \\times LOC', 'C \\times Cap', 'C^2 \\times Cap',
         'C \\times RPM', 'C^2 \\times RPM', 'B \\times LOC', 'B \\times Cap', 'B \\times RPM',
         'A \\times LOC', 'A \\times Cap', 'A \\times RPM', 'LOC \\times Cap', 'LOC \\times RPM',
         'Cap \\times RPM']
terms = [f'${t}$' for t in terms]

True

#### Design combination

In [3]:
design_comb = pd.read_excel(f'..\\assets\\design_comb.xlsx', header=None, usecols=range(3,10))
design_comb.columns = columns
X_comb = x2fx(design_comb.to_numpy(), model_enc)
M_comb = X_comb.T @ np.linalg.solve(V, X_comb)

det_val = np.linalg.det(M_comb)**(1/len(model))
print(det_val)

11.570656784896952


#### Design optim

In [4]:
design_optim = pd.read_excel(f'..\\assets\\design_optim.xlsx')
X_optim = x2fx(design_optim.to_numpy(), model_enc)
M_optim = X_optim.T @ np.linalg.solve(V, X_optim)

det_val = np.linalg.det(M_optim)**(1/len(model))
print(det_val)

12.72672630249043


In [5]:
extr = np.arange(len(M_optim)), np.arange(len(M_optim))
Mratio = np.linalg.inv(M_comb)[extr] / np.linalg.inv(M_optim)[extr]

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(3, 1)
fig.add_trace(go.Heatmap(
    z=Mratio[:14][np.newaxis, :],
    text=Mratio[:14][np.newaxis, :],
    texttemplate='%{text:.2f}',
    coloraxis='coloraxis',
    x=terms[:14]
), 1, 1)
fig.add_trace(go.Heatmap(
    z=Mratio[14:20][np.newaxis, :],
    text=Mratio[14:20][np.newaxis, :],
    texttemplate='%{text:.2f}',
    coloraxis='coloraxis',
    x=terms[14:20]
), 2, 1)
fig.add_trace(go.Heatmap(
    z=Mratio[20:][np.newaxis, :],
    text=Mratio[20:][np.newaxis, :],
    texttemplate='%{text:.2f}',
    coloraxis='coloraxis',
    x=terms[20:]
), 3, 1)
fig.update_layout(
    coloraxis={'colorscale': 'viridis'},
    margin=dict(
        l=10, r=0, b=0, t=10
    )
)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(tickangle=0, tickfont=dict(size=10))
fig.update_xaxes(tickangle=45, row=3, col=1)

fig.write_image('../figures/variance_ratios.png')

In [18]:
extr = np.arange(len(M_optim)), np.arange(len(M_optim))
M_comb_extr = np.linalg.inv(M_comb)[extr]
M_optim_extr = np.linalg.inv(M_optim)[extr]

table = '\\\\\n\\hline\n'.join([' & '.join(f'{y:.3f}' if isinstance(y, float) else str(y) for y in x) 
                                for x in zip(terms, M_comb_extr, M_optim_extr, M_comb_extr / M_optim_extr)])
print(table)

$icept$ & 1.547 & 1.684 & 0.919\\
\hline
$D$ & 0.784 & 0.776 & 1.011\\
\hline
$C$ & 0.406 & 0.388 & 1.044\\
\hline
$C^2$ & 1.172 & 1.192 & 0.983\\
\hline
$D \times C$ & 0.416 & 0.421 & 0.987\\
\hline
$B$ & 0.318 & 0.284 & 1.118\\
\hline
$A$ & 0.340 & 0.296 & 1.147\\
\hline
$D \times B$ & 0.103 & 0.093 & 1.114\\
\hline
$D \times A$ & 0.097 & 0.094 & 1.028\\
\hline
$C \times B$ & 0.169 & 0.171 & 0.991\\
\hline
$C^2 \times B$ & 0.497 & 0.474 & 1.048\\
\hline
$C \times A$ & 0.164 & 0.166 & 0.988\\
\hline
$C^2 \times A$ & 0.499 & 0.470 & 1.061\\
\hline
$B \times A$ & 0.099 & 0.094 & 1.053\\
\hline
$LOC$ & 0.098 & 0.073 & 1.339\\
\hline
$LOC^2$ & 0.184 & 0.193 & 0.952\\
\hline
$Cap$ & 0.097 & 0.082 & 1.185\\
\hline
$Cap^2$ & 0.187 & 0.199 & 0.941\\
\hline
$RPM$ & 0.097 & 0.079 & 1.229\\
\hline
$RPM^2$ & 0.233 & 0.229 & 1.016\\
\hline
$D \times LOC$ & 0.040 & 0.025 & 1.623\\
\hline
$D \times Cap$ & 0.048 & 0.025 & 1.944\\
\hline
$D \times RPM$ & 0.042 & 0.025 & 1.710\\
\hline
$C \times LOC$ &